## 演習問題１

何らかのテキストデータを収集し、その前処理を行いLSTMで文章生成を学習させよ。LSTMブロックを２〜３個積層したモデルも試してみよ。色々工夫して結果の改善具合を検討できるとなお良い。特に学習済みの埋め込み層を用いてみよ。

In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget4 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget4 libencode-locale-perl libfcgi-pe

In [3]:
import numpy as np
import MeCab
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, GRU, LSTM, Embedding
from tensorflow.keras.optimizers import RMSprop
from google.colab import drive

In [4]:
drive.mount('/content/drive')
home_path = '/content/drive/My Drive/DeepLearning/Report03/'

Mounted at /content/drive


## デート処理


### ダウンロード

作品名：	愛の終り(あいのおわり)

著者名：	山川 方夫　

https://www.aozora.gr.jp/cards/001801/card59967.html#download

In [5]:
file_path = home_path + 'aino_owari.txt'
    
with open(file_path, 'r', encoding='shift-jis') as text_file:
    strings = text_file.read()
    
text = strings.splitlines()

In [6]:
text[:10]

['愛の終り',
 '山川方夫',
 '',
 '-------------------------------------------------------',
 '【テキスト中に現れる記号について】',
 '',
 '《》：ルビ',
 '（例）頬笑《ほほえ》',
 '',
 '［＃］：入力者注\u3000主に外字の説明や、傍点の位置の指定']

In [7]:
len(text)

120

### 前処理

In [8]:
import re
import codecs
    
end = 0
# 末尾の情報削除
for i in range(20):
    if text[-i][:3]=='底本：':
        end=i+3
text = text[:-end]

# 本文以前の冒頭を削除     
for i, p in enumerate(text):
    if p[:4]=='----':
        end = i+1
text = text[end:]

# 見出し、ルビ等削除
text_processed = ''
marks = re.compile(r'［[^［]*］|《[^《]*》|｜|\u3000') # ぜひ一度は正規表現の基礎を勉強すること！
for p in text:
    q = re.sub(marks, '', p)
    text_processed += q

In [9]:
text = text_processed
text

'ドアが開くと、一人の青年が入ってきた。青年は流行のシルキイな布地の背広をぴったりと身につけ、暗緑色のサン・グラスで顔をかくしていた。そのまま、彼はすばやく室内をながめた。彼女はゆっくりとベッドから起き上った。「いらっしゃい、坊や」青年はサン・グラスをはずした。色白で女性的な面だちの、ただ眉だけが濃いその顔に、いかにも人工的な、不自然な微笑がうかんだ。彼はつかつかと歩み寄って彼女を抱き、接吻した。彼女はかるい呻き声をもらした。「ねえ、冗談なんでしょう？いまの電話」同じつくりつけたような微笑のまま、青年はいった。彼女は、部屋の中央に歩いた。そのテーブルには、すでに半分ほどになったコニャックの瓶と、二つのグラスがあった。「……真白なスポーツ・カーを買ったんですって？」彼女は、それが癖らしい疲れたような声でいった。「すてきね。でも、ここに乗りつけたのはどうやら違う車だったようね。私、この窓から見てたの」彼女はレースのカーテンの下りた窓を背にして、コニャックを掌で温めながら笑いかけた。「でも、よく来てくれたわ。私、もう来てくれないんじゃないかって考えていたの」「そんなこと……、ぼくの愛しているのは先生だけだってこと、先生だって知っているはずじゃないの」青年は甘えるような表情をつくった。が、その声にはあきらかに演技が感じられた。「そうちょいちょい逢いにくるわけにもいかないし、目立つような真似はできない。ぼくだって苦しいんだ。……先生だけは、わかって下さると思っていた」「そうね、わかってるわ。よく」彼女は頬笑みながら、うっとりとしたようにその青年をみつめた。「ほんとにわかっていてくれるの？」「ええ。ほんとにわかっているつもりよ」「じゃ、どうしてさっきの電話みたいなこと、いったりなんかするの？」青年は絨毯にひざまずいて、彼女の手を握った。「ね。いってよ、ただのおどかしなんでしょ？そうなんでしょ？」「だいぶ、おいそぎのようね」彼女は笑いながら答えた。「これから、すぐ飛行機で大阪に……」「そう、それは結構ね。いやみじゃなく、おめでとうをいわせてもらうわ」青年は黙った。わざとそわそわと腕時計をみた。だが、彼女はソファに腰を下ろし、ネグリジェの脚を組んで、青年のその落着きのなさ、焦立ちを完全に無視していた。彼女はしゃべりだした。「昨夜のテレビみたわ。あなたが初出場したＮ・Ｈ・Ｋの紅白歌合

In [10]:
title = "愛の終り。"
text = title + text

In [11]:
text

'愛の終り。ドアが開くと、一人の青年が入ってきた。青年は流行のシルキイな布地の背広をぴったりと身につけ、暗緑色のサン・グラスで顔をかくしていた。そのまま、彼はすばやく室内をながめた。彼女はゆっくりとベッドから起き上った。「いらっしゃい、坊や」青年はサン・グラスをはずした。色白で女性的な面だちの、ただ眉だけが濃いその顔に、いかにも人工的な、不自然な微笑がうかんだ。彼はつかつかと歩み寄って彼女を抱き、接吻した。彼女はかるい呻き声をもらした。「ねえ、冗談なんでしょう？いまの電話」同じつくりつけたような微笑のまま、青年はいった。彼女は、部屋の中央に歩いた。そのテーブルには、すでに半分ほどになったコニャックの瓶と、二つのグラスがあった。「……真白なスポーツ・カーを買ったんですって？」彼女は、それが癖らしい疲れたような声でいった。「すてきね。でも、ここに乗りつけたのはどうやら違う車だったようね。私、この窓から見てたの」彼女はレースのカーテンの下りた窓を背にして、コニャックを掌で温めながら笑いかけた。「でも、よく来てくれたわ。私、もう来てくれないんじゃないかって考えていたの」「そんなこと……、ぼくの愛しているのは先生だけだってこと、先生だって知っているはずじゃないの」青年は甘えるような表情をつくった。が、その声にはあきらかに演技が感じられた。「そうちょいちょい逢いにくるわけにもいかないし、目立つような真似はできない。ぼくだって苦しいんだ。……先生だけは、わかって下さると思っていた」「そうね、わかってるわ。よく」彼女は頬笑みながら、うっとりとしたようにその青年をみつめた。「ほんとにわかっていてくれるの？」「ええ。ほんとにわかっているつもりよ」「じゃ、どうしてさっきの電話みたいなこと、いったりなんかするの？」青年は絨毯にひざまずいて、彼女の手を握った。「ね。いってよ、ただのおどかしなんでしょ？そうなんでしょ？」「だいぶ、おいそぎのようね」彼女は笑いながら答えた。「これから、すぐ飛行機で大阪に……」「そう、それは結構ね。いやみじゃなく、おめでとうをいわせてもらうわ」青年は黙った。わざとそわそわと腕時計をみた。だが、彼女はソファに腰を下ろし、ネグリジェの脚を組んで、青年のその落着きのなさ、焦立ちを完全に無視していた。彼女はしゃべりだした。「昨夜のテレビみたわ。あなたが初出場したＮ・Ｈ・Ｋ

### 形態素解析

In [12]:
import MeCab

tagger = MeCab.Tagger('-Owakati')

result = tagger.parse(text)
wakachi = result.split()

wakachi[:10]

['愛', 'の', '終り', '。', 'ドア', 'が', '開く', 'と', '、', '一']

In [13]:
len(wakachi)

3426

In [14]:
chars = sorted(list(set(wakachi)))

print('全文字数:', len(chars))

全文字数: 767


### トークナイザ

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(wakachi)

seq = tokenizer.texts_to_sequences(wakachi)
seq = np.array(seq) -1 # 0から始めるようにシフト
seq = np.squeeze(seq)

In [16]:
tokenizer.index_word[99]

'ながら'

In [17]:
tokenizer.word_index["ながら"]

99

In [18]:
tokenizer.texts_to_sequences(["愛"])

[[280]]

In [19]:
tokenizer.sequences_to_texts([[280]])

['愛']

### 訓練データ生成

In [20]:
maxlen = 16
step = 3
sentences = []
next_chars = []

for i in range(0, len(seq) - maxlen, step):
    sentences.append(seq[i:i+maxlen])
    next_chars.append(seq[i+maxlen])

x_train = np.array(sentences)
next_chars = np.array(next_chars)

In [21]:
print(x_train.shape)
print(next_chars.shape)

(1137, 16)
(1137,)


In [22]:
x_train[0]

array([279,   2, 280,   1, 113,   9, 281,  12,   0, 114, 155,   2,  19,
         9, 282,   6])

In [23]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(next_chars, num_classes=len(chars))

## モデル構成

### モデル構築
双方向LSTM、２層を使う

In [24]:
from pickle import TRUE
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import RMSprop

def buildModel():
  model = Sequential()
  model.add(Embedding(input_dim=len(chars), output_dim=16, input_length=maxlen))
  model.add(Bidirectional(LSTM(128, 
                                recurrent_initializer="glorot_uniform", 
                                return_sequences = TRUE,
                                recurrent_dropout=0.25)))
  model.add(Bidirectional(LSTM(128,  
                                recurrent_initializer="glorot_uniform", 
                                return_sequences = False,
                                recurrent_dropout=0.25)))
  
  model.add(Dense(len(chars), activation='softmax'))

  return model

model = buildModel()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 16, 16)            12272     
                                                                 
 bidirectional (Bidirectiona  (None, 16, 256)          148480    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 767)               197119    
                                                                 
Total params: 752,111
Trainable params: 752,111
Non-trainable params: 0
_________________________________________________________________


In [25]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### コールバック

In [28]:
import random
import sys

# 様々な温度の予測確率分布からランダムに予測文字をサンプリングすることで、生成文の多様性を変える
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = preds ** (1/temperature)
    preds = preds / np.sum(preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 今回は生成のseedは固定
start_index = random.randint(0, len(wakachi) - maxlen - 1)
sentence = wakachi[start_index: start_index + maxlen]

def on_epoch_end(epoch, logs):
    print()
    print('Epoch: %d' % (epoch+1), '-'*30)

    generated = ''
    sentence = wakachi[start_index: start_index + maxlen]
    sys.stdout.write(generated)

    for i in range(70):
        x_pred = np.zeros((1, maxlen))
        for t, char in enumerate(sentence):
            char_idx = tokenizer.texts_to_sequences([char])
            char_idx = char_idx[0][0]
            x_pred[0, t] = (char_idx-1) # インデックスを1ずらしたことに注意

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, 0.8)
        next_char = tokenizer.sequences_to_texts([[next_index+1]])[0]

        generated += next_char
        sentence = sentence[1:] + [next_char]

        sys.stdout.write(next_char)
        sys.stdout.flush()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

### 学習

In [29]:
model.fit(x_train, y_train,
          batch_size=16,
          epochs=50,
          callbacks=[print_callback])

Epoch 1/50
72/72 [==============================] - ETA: 0s - loss: 3.5683
Epoch: 1 ------------------------------
72/72 [==============================] - 19s 266ms/step - loss: 3.5683
Epoch 2/50
72/72 [==============================] - ETA: 0s - loss: 3.1984
Epoch: 2 ------------------------------
72/72 [==============================] - 19s 271ms/step - loss: 3.1984
Epoch 3/50
72/72 [==============================] - ETA: 0s - loss: 2.8390
Epoch: 3 ------------------------------
72/72 [==============================] - 19s 271ms/step - loss: 2.8390
Epoch 4/50
72/72 [==============================] - ETA: 0s - loss: 2.4732
Epoch: 4 ------------------------------
72/72 [==============================] - 19s 266ms/step - loss: 2.4732
Epoch 5/50
72/72 [==============================] - ETA: 0s - loss: 2.1004
Epoch: 5 ------------------------------
72/72 [==============================] - 19s 270ms/step - loss: 2.1004
Epoch 6/50
72/72 [==============================] - ETA: 0s - loss: 1.

## 文章生成

### 汎用メソッド

In [30]:
diversity = 0.8

def generatSentence(generated, sentence):
  for i in range(10):
    x_pred = np.zeros((1, maxlen))
    for t, char in enumerate(sentence):
      char_idx = tokenizer.texts_to_sequences([char])
      char_idx = char_idx[0][0]
      x_pred[0, t] = (char_idx-1)

      preds = model.predict(x_pred, verbose=0)[0]
      next_index = sample(preds, diversity)
      next_char = tokenizer.sequences_to_texts([[next_index+1]])[0]

      generated += next_char
      sentence = sentence[1:] + [next_char]
  
  return generated

### テストデータ１

In [37]:
# テストデータ
text_val = '愛'

tagger = MeCab.Tagger('-Owakati')

result_val = tagger.parse(text_val)
wakachi_val = result_val.split()

generated = text_val
sentence = wakachi_val

generatSentence(generated, sentence)

'愛それ青年あなたゆるし彼女あなたゆるし彼女わそれ'

### テストデータ２

In [38]:
# テストデータ
text_val = '終り'

tagger = MeCab.Tagger('-Owakati')

result_val = tagger.parse(text_val)
wakachi_val = result_val.split()

generated = text_val
sentence = wakachi_val

generatSentence(generated, sentence)

'終りわかっ青年あなたゆるし四。そう青年あなたそっくり'

### テストデータ３

In [39]:
# テストデータ
text_val = '愛の終り'

tagger = MeCab.Tagger('-Owakati')

result_val = tagger.parse(text_val)
wakachi_val = result_val.split()

generated = text_val
sentence = wakachi_val

generatSentence(generated, sentence)

'愛の終りそうそれそれのあなた窓彼女彼女そんなにわ調子あなた青年今日それあなたあなたあなたゆるし「あげ彼女彼女しあなたあなた窓ゆるしゆるしそっくり'